In [69]:
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [70]:
# load & read file
vacation_df = pd.read_csv("Data/WeatherPy_vacation.csv")
vacation_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,80.60,scattered clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
1,Los Llanos De Aridane,ES,77.00,broken clouds,28.66,-17.92,Valle Aridane
2,Hithadhoo,MV,84.06,overcast clouds,-0.60,73.08,Scoop Guest House
3,Kalangala,UG,71.60,few clouds,-0.31,32.23,Ssese Habitat Resort
4,Albany,US,78.80,scattered clouds,42.60,-73.97,NaN
...,...,...,...,...,...,...,...
244,Camacha,PT,71.60,overcast clouds,33.08,-16.33,Hotel Porto Santo & Spa
245,Lodja,CD,75.54,clear sky,-3.48,23.43,NaN
246,Belmonte,BR,80.60,scattered clouds,-15.86,-38.88,Pousada Monte Carmelo O Pão
247,Kalemie,CD,70.59,clear sky,-5.95,29.19,Mahobilali


In [71]:
# create dataframes for 4 cities
barcelos_df = vacation_df[vacation_df["City"] == "Barcelos"]
lata_df = vacation_df[vacation_df["City"] == "Lata"]
aljezur_df = vacation_df[vacation_df["City"] == "Aljezur"]
muros_df = vacation_df[vacation_df["City"] == "Muros"]
muros_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
38,Muros,ES,78.01,few clouds,42.78,-9.06,Apartment Av. Castelao


In [86]:
# function to get tuple of lat & long for each city
def coordinates(city, city_df):
    city = {}
    for index, row in city_df.iterrows():
        lat = row["Lat"]
        lng = row["Lng"]
        city["location"] = (lat, lng)
    return city["location"]

# barcelos
barcelos = coordinates(barcelos, barcelos_df)

# lata
lata = coordinates(lata, lata_df)

# aljezur
aljezur = coordinates(aljezur, aljezur_df)

# muros
muros = coordinates(muros, muros_df)

In [90]:
# create directions layer map
import gmaps
import gmaps.datasets
gmaps.configure(api_key=g_key)

# latitude-longitude pairs
barcelos = barcelos
lata = lata
aljezur = aljezur
muros = muros

fig = gmaps.figure()
muros2aljezur_via_barcelos_lata = gmaps.directions_layer(muros, aljezur,
                                                         waypoints=[barcelos, lata],
                                                         travel_mode='DRIVING')
fig.add_layer(muros2aljezur_via_barcelos_lata)
fig

Figure(layout=FigureLayout(height='420px'))

In [64]:
# concatenate cities into 1 dataframe
cities_df = pd.concat([barcelos_df, lata_df, aljezur_df, muros_df], axis=0)
cities_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
187,Barcelos,PT,77.00,broken clouds,41.54,-8.62,Hotel Do Terço
43,Lata,PT,70.00,overcast clouds,40.16,-8.33,Casa Maquia
126,Aljezur,PT,79.00,clear sky,37.32,-8.80,Amazigh Hostel
38,Muros,ES,78.01,few clouds,42.78,-9.06,Apartment Av. Castelao


In [91]:
# create marker layer map for the route

# info box template for map markers
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# store cities_df row in info box
hotel_info = [info_box_template.format(**row) for index, row in cities_df.iterrows()]

# add pop-up markers for each city on the route
locations = cities_df[["Lat", "Lng"]]
max_temp = cities_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))